<a href="https://colab.research.google.com/github/chni9/MyDataPortfolio/blob/main/Automatic_mailing_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **How to use ?**



**To send automatic emails to our partners :**

1.   Click on the "Run time" menu
2.   Click "Run all"
3.   Allow connections to your Google and Google Drive (you may need to insert your password)
4.   Check below if the mails have succesfully been sent






# **Link partner <-> news**

## **Extract**

**Define needed variables and parameters**

In [ ]:
#Link of "Partner preferences" spreadsheet
link_partner_pref = 'https://docs.google.com/spreadsheets/d/1BXwiMpDs3TeiOIsNnyTF-VSmY1QsokEIz2CoN7CdxJ8/edit'
#Link of "Categorization" spreadsheet
link_categorization = 'https://docs.google.com/spreadsheets/d/1rmNzama38ejksRAsDBumZCP6Hj5zHyirm1piZ7NsDts/edit#gid=0'
#Link of "Link partner news" spreadsheet
link_partner_news = 'https://docs.google.com/spreadsheets/d/1c55VVr3aII3Itf2G2686zjb21v4a9b1LR8eytZsTUKQ/edit#gid=0'

#This list is the list of columns of our partners dataframe
new_columns_partners = [
    "first_name1",
    "last_name1",
    "email1",
    "company",
    "first_name2",
    "last_name2",
    "email2",
    "medical_pathway",
    "medical_speciality",
    "technology",
    "demographic",
    "geography",
    "business_model",
    "company_level",
    "other_interests",
    "date",
    "token"]

#This list is the list of columns of our news/startups dataframe
new_columns_news = [
    "startup",
    "country",
    "funding",
    "news",
    "category"
]

**Library imports and connection to google sheets**

In [ ]:
# Connect to server
import smtplib
# Manipulate DataFrames
import pandas as pd
# Google Spreadsheets client
import gspread
# Google Spreadsheets client to export dataframes to G Sheets
import gspread_dataframe as gd
# Authenticate with Google
from google.auth import default
# Authenticate with Google Colab
from google.colab import auth
# Connecting the drive to the Google Colab
from google.colab import drive


# Connect to the Google Drive API
from googleapiclient.discovery import build
# Error handling of Google Drive imports
from googleapiclient.errors import HttpError

# Build the email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import ast # To delete when cleaning the code

# Used to make an API call
import requests

**Connecting my Google account to the notebook**

In [ ]:
#Authentification of the user
auth.authenticate_user()

#Open credential of Google
creds, _ = default()

#Connect Google Sheets to our notebook
gc = gspread.authorize(creds)

**Import of our Google Sheets**

In [ ]:
# Open our spreadsheet of partners
worksheet = gc.open_by_url(link_partner_pref).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame
df_partners = pd.DataFrame.from_records(rows[1:],
                              columns=new_columns_partners)
df_partners.index.name = "partner_id"

In [ ]:
# Open our spreadsheet of news
worksheet = gc.open_by_url(link_categorization).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame
df_news = pd.DataFrame.from_records(rows[1:],
                                    columns=new_columns_news)
df_news.index.name = "news_id"

#Delete all '\n' in the 'news' column
df_news['news'] = df_news['news'].str.replace('\n', '')


## **Transform**

**Link each partner to the news/startups of interest**

In [ ]:
#Create an empty list to fill
linked_news = pd.Series([None for k in range(df_news.shape[0])])

# Link each partner to a startup that is in the same medical speciality
for partner_id, speciality in enumerate(df_partners.medical_speciality):
  linked_news[partner_id] = [news_id for news_id, category in enumerate(df_news['category']) if category in speciality]

# Create a dataframe with each partner linked to news_id
df_partners['news_id'] = linked_news

**Separating contact 1 and contact 2 into two separate rows**

In [ ]:
# Assuming df is your existing dataframe
df1 = df_partners.copy()
df2 = df_partners[['first_name2', 'last_name2', 'email2', 'company',
          'medical_pathway', 'medical_speciality', 'technology', 'demographic',
          'geography', 'business_model', 'company_level', 'other_interests',
          'date', 'token', 'news_id']].copy()

# Rename columns in the second dataframe
df2.rename(columns={
    'first_name2': 'first_name1',
    'last_name2': 'last_name1',
    'email2': 'email1'
}, inplace=True)

# Drop the '2' columns from the first dataframe
df1.drop(['first_name2', 'last_name2', 'email2'], axis=1, inplace=True)

# Filter out rows from df2 where 'first_name1', 'last_name1', 'email1' are empty
df2 = df2.loc[(df2['first_name1'] != '') & (df2['last_name1'] != '') & (df2['email1'] != '')]

# Append the second dataframe to the first dataframe
df_partners = df1.append(df2)

# Reset the index
df_partners.reset_index(drop=True, inplace=True)

df_partners.head(3)

<ipython-input-41-0c4f90a57bd2>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_partners = df1.append(df2)


,first_name1,last_name1,email1,company,medical_pathway,medical_speciality,technology,demographic,geography,business_model,company_level,other_interests,date,token,news_id
0,Yassine,Chenik,yassine.chenik@ehealthventure.com,EHV,Health Education,"Oncology, Nutrition",Blockchain & Crypto,Seniors,Europe,Hospitals & Care Providers,Growth (Series C+),,11/04/2023 15:38:09,t5y9cm7hmrxz9mlmnvt5y9cb3k0c9k4e,"[6, 7, 8, 11, 14, 35, 39, 43, 44, 51]"
1,Seamus,Carey,sca@alt-f1.be,Alt-F1,"Prevention, Screening, Health Education, Diagn...","Cardiology, Sports Medicine, Gastroenterology,...","Augmented & Virtual Reality, Artificial Intell...","Children, Adults, Seniors","Belgium, Europe","Employers, Direct to Consumer/Patient, Hospita...","Venture-Building, Start-Up (Pre-Seed & Seed), ...",,03/05/2023 12:38:14,i1hu4xnzxwgrrjpfbh0li1huzwut011l,"[12, 15, 18, 27, 28, 30, 36, 51, 54, 56, 62, 7..."
2,Contact,EHV,contact@ehealthventure.com,E-Health Venture,,"Physical Medicine & Rehabilitation, Ophthalmol...",,,,,,,17/04/2023 15:37:30,enk2ctzjtyqmp3ahenk2ctj0g3vogs4y,"[1, 2, 3, 4, 45, 51, 63, 65]"


In [ ]:
# Creating a 'tags' column which is a merge of all preferences columns
df_partners['tags'] = df_partners[['medical_pathway', 'medical_speciality', 'technology',
                 'demographic', 'geography', 'business_model',
                 'company_level']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

## **Load**

**Export of our tables into GSheets**

In [ ]:
#Export the dataframe into a google sheets

#Open a new sheet
sheet1 = gc.open_by_url(link_partner_news)
sheet_name1 = "Partners"
worksheet = sheet1.worksheet(sheet_name1)
#Export our "df_partner_linked" dataframe into our worksheet
gd.set_with_dataframe(worksheet, df_partners)


print('Data uploaded to Google Sheets!')

Data uploaded to Google Sheets!


# **Automatic mailing**

## **Pre-processing**

**Extracting the needed informations for the mail (startups and news)**

In [ ]:
all_mail_informations = []
#Message to notify if the mail has been sent succesfully
success_message = ""

for partner_id in df_partners.index:
  one_mail_informations = []

  #Extracting Name and Email address
  one_mail_informations.append(("first_name",df_partners.loc[partner_id, "first_name1"]))
  one_mail_informations.append(("address",df_partners.loc[partner_id, "email1"]))

  #Extracting 3 startups, categories and descriptions (news)
  if len(df_partners.loc[partner_id, "news_id"]) >= 3:
    for i, news_id in enumerate(df_partners.loc[partner_id, "news_id"], start=1):
      if i <= 3:
          one_mail_informations.extend([(f"startup{i}", df_news.loc[news_id, "startup"]),
                                        (f"category{i}", df_news.loc[news_id, "category"]),
                                        (f"news{i}", df_news.loc[news_id, "news"])])
  else:
    success_message += f"Mail not sent to {df_partners.loc[partner_id, 'email1']}. Not enough matched news/startups\n"

  #Extracting the list of unique categories of interest and formatting it into a natural sentence
  unique_categories = df_partners.loc[partner_id, "medical_speciality"].split(", ")
  list_category = unique_categories[0] if len(unique_categories) == 1 else " and ".join([", ".join(unique_categories[:-1]), unique_categories[-1]]) if len(unique_categories) > 1 else ""
  one_mail_informations.append(('list_category',list_category))

  #Transforming th mail informations into a dictionnary
  one_mail_informations = dict(one_mail_informations)

  #Adding the mail informations in the list of all the mails informations
  all_mail_informations.append(one_mail_informations)

**API call**

In [ ]:
all_html = list()
for partner_id in df_partners.index:
  # Extracting the tags and company of each partner
  tags = df_partners.loc[partner_id, "tags"]
  company = df_partners.loc[partner_id, "company"]
  name = df_partners.loc[partner_id, "first_name1"]

  # Remove the commas from the string
  tags = tags.replace(",", "")

  url = 'https://stg-dot-europeangrantstenders.ew.r.appspot.com/api/calls'
  params = {
      'companyId': company,
      'tags': tags,
      'recipientName': name
  }

  response = requests.get(url, params=params)

  # Check the status code to ensure the request was successful (200)
  if response.status_code == 200:
      all_html.append(response.text)
  else:
      print('Error occurred:', response.status_code)

## **Automatic newsletter**

**Mailjet API installation**

In [ ]:
#Installing the mailjet module for python
!pip install mailjet_rest

#Importing the mailjet Client
from mailjet_rest import Client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Connection to Mailjet**

In [ ]:
# Public API key of the account sending mails on Mailjet
api_key = '9c070e588a0ec0119c8243cfb0b451fe'
# Private API
api_secret = 'eae24fcdd9e26b0bf66c3a25ed02a0ea'

In [ ]:
#Connection to the Mailjet client
mailjet = Client(auth=(api_key, api_secret), version='v3.1')

**Building the emails to send (startups & news)**

In [ ]:
all_emails = list()
for i, variables in enumerate(all_mail_informations):
  if i < len(all_html):

    # News and startups

    # all_emails.append({
    #       'Messages': [
    #         {
    #           "From": {
    #             "Email": "contact@ehealthventure.com",
    #             "Name": "Marius Declerck"
    #           },
    #           "To": [
    #             {
    #               "Email": variables['address'],
    #               "Name": variables['first_name']
    #             }
    #           ],
    #           "TemplateID": 4723481,
    #           "TemplateLanguage": True,
    #           "Subject": "EHV Newsletter",
    #           "Variables": variables
    #         }
    #       ]
    #     })

    # Calls

    all_emails.append({
    'Messages': [
          {
              "From": {
                  "Email": "contact@ehealthventure.com",
                  "Name": "Séamus Carey"
              },
              "To": [
                  {
                      "Email": variables['address'],
                      "Name": variables['first_name']
                  }
              ],
              "HTMLPart": all_html[i],
              "Subject": "E-Health Venture Newsletter",
              "Variables": {
								"Name": variables['first_name']
						}
          }
        ]
      })

**Sending the email**

In [ ]:
success_message = ""
#Sending all the emails
for email in all_emails:
  result = mailjet.send.create(data=email)
  success_message += f"Mail sent successfully to {email['Messages'][0]['To'][0]['Email']}\n"

# **Check here if mails have been sent**

In [ ]:
print(success_message)

Mail sent successfully to yassine.chenik@ehealthventure.com
Mail sent successfully to sca@alt-f1.be
Mail sent successfully to contact@ehealthventure.com
Mail sent successfully to abo+ehv_newsletter@alt-f1.be
Mail sent successfully to jason.pettiaux@ehealthventure.com
Mail sent successfully to aam@alt-f1.be

